# Interpreter

## O que é?

O padrão _Interpreter_, prevê a criação de uma "gramática" intuitiva para operações comuns e bem definidades, asssim como um _interpreter_ para que consiga traduzir e executar tal "gramática".

## Por quê?

Muitas vezes necessário realizar grandes sequências de operações repetidas, o que pode tornar o código pouco legível ou cansativo de escrever.

Exemplo:
    Escrever `(1 + 2 / 3) ** 2 ` é muito mais fácil de ler e escrever do que `pow(sum(1, div(2, 3), 2)`
    
Isso também permite que especialistas que não necessariamente sabem programar possam interagir com o sistema e compor operações complexas (e.g.: SQL).

## Uso:

Este padrão é mais eficaz quando:
- Há um número limitado de expressões para serem para evitar que gramática torne-se muito complexa.
- Há um grupo de operações que são utilizadas muito frequentemente e devem ser compostas.
- Não programadores precisam realizar operações complexas ad-hoc.

## Estrutura: 

![struct](https://upload.wikimedia.org/wikipedia/commons/2/21/Interpreter_design_pattern.png)

## Exemplo:

Um exemplo simples do uso do _Chain of Responsibility_ eh uma calculadora. Cada vez que voce usa uma calculadora voce envia uma requisição de um calculo, um objeto composto por dois numeros e uma operacao, esse calculo pode ser executado por diversos objetos (dependendo da operacao desejada)

In [ ]:
1 + 2 ** 2 / 3

In [21]:
import abc

class Context:
    """
    Holds information global information about the expression to be decoded.
    """
    def __init__(self, variables):
        self._variables = variables or {}
        
    def lookup_var(self, symbol):
        return self._variables.get(symbol)


class AbstractExpression(abc.ABC):
    """
    Expressions that compose a language.
    """
    
    @abc.abstractmethod
    def interpret(self, context):
        return 

    
class TerminalExpression(AbstractExpression):
    """
    Expressions that doesn't depend on other expressions.
    """

    def __init__(self, symbol):
        self._symbol = symbol
        
    def __repr__(self):
        return self._symbol
       
    
class Literal(TerminalExpression):
    """
    Literal value.
    """
    def interpret(self, context):
        return eval(self._symbol)
    

class Variable(TerminalExpression):
    """
    Variable to be replaced.
    """
    def interpret(self, context):
        return context.lookup_var(self._symbol)
    
    
class NonTerminalExpression(AbstractExpression):
    """
    Expressions that cotain other expressions.
    """
    def __init__(self, exp1, exp2):
        self._exp1 = exp1
        self._exp2 = exp2
    
    def __repr__(self):
        if hasattr(self, "_symbol"):
            return f"{self._exp1} {self._symbol} {self._exp2}"
        else:
            return f"{self.__class__.__name__}({self._exp1}, {self._exp2})"
    
    
class Sum(NonTerminalExpression):
    _symbol = "+"
    
    def interpret(self, context):
        return self._exp1.interpret(context) + self._exp2.interpret(context)
    

class Diff(NonTerminalExpression):
    _symbol = "-"
    
    def interpret(self, context):
        return self._exp1.interpret(context) - self._exp2.interpret(context)
    

class Mult(NonTerminalExpression):
    _symbol = "*"
    
    def interpret(self, context):
        return self._exp1.interpret(context) * self._exp2.interpret(context)
    
    
class Div(NonTerminalExpression):
    _symbol = "/"
    
    def interpret(self, context):
        return self._exp1.interpret(context) / self._exp2.interpret(context)
    
    
class Pow(NonTerminalExpression):
    _symbol = "**"
        
    def interpret(self, context):
        return self._exp1.interpret(context) ** self._exp2.interpret(context)

Vamos escrever a expressão que nós hadviamos mencionado anteriormente: `1 + x ** 4 / 3 * 4`, send `x = 5`

In [22]:
ast = Sum(Literal("1"), Mult(Div(Pow(Variable("x"), Literal("4")), Literal("3")), Literal("4")))
ast

1 + x ** 4 / 3 * 4

Para `x = 5`...

In [23]:
ctx = Context({"x": 5})

ast.interpret(ctx)

834.3333333333334

Comparando com a resposta do Python...

In [25]:
x = 5

1 + x ** 4 / 3 * 4

834.3333333333334

Claro, foi um trabalho da maldito escrever isso a mão, mas com um bom _parser_ a AST tornaria o processo muito simples.

## Prós e contras:

### Prós

- Torna expressões complexas mais fáceis de escrever e mais legíveis. (e.g.: usar um compilador ao invés de escrever em assembly ou machine code).
- Adicionar novas operações é simples, basta criar uma nova subclasse de `AbstractExpression`.
- Como a gramática abstrai a implementação das expressões, mudanças na lógica de código tende a ser simples de implementar.


### Contras

- A implementação desse padrão pode ser bastante complexa.
- A linguagem deve ser aprendida pelos usuários e mantida pelos desenvolvedores.


## Discussao:

As the note says in Known Uses, Interpreter is most often used "in compilers implemented in object-oriented languages...". What are other uses of Interpreter and how do they differ from simply reading in a stream of data and creating some structure to represent that data? 
 